# 4. Modélisation avec Régression Logistique

Ce notebook se concentre sur la quatrième étape du processus d'analyse prédictive : la modélisation avec régression logistique. Nous allons développer un modèle de régression logistique pour prédire le statut de crédit des clients, l'évaluer sur le jeu de test, et analyser ses performances.

## 4.1 Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# Bibliothèques pour la modélisation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Pour une meilleure lisibilité des graphiques
%matplotlib inline

# Configuration de l'aléatoire pour la reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 4.2 Chargement des données préparées pour la modélisation

In [ ]:
# Chemin vers les données préparées pour la modélisation
model_data_dir = "/home/ubuntu/notebooks/model_data"

# Vérification de l'existence du dossier
if os.path.exists(model_data_dir):
    print(f"Le dossier existe à l'emplacement : {model_data_dir}")
    
    # Chargement des ensembles d'entraînement et de test
    X_train = joblib.load(os.path.join(model_data_dir, "X_train.pkl"))
    X_test = joblib.load(os.path.join(model_data_dir, "X_test.pkl"))
    y_train = joblib.load(os.path.join(model_data_dir, "y_train.pkl"))
    y_test = joblib.load(os.path.join(model_data_dir, "y_test.pkl"))
    
    # Chargement des ensembles standardisés
    X_train_scaled = joblib.load(os.path.join(model_data_dir, "X_train_scaled.pkl"))
    X_test_scaled = joblib.load(os.path.join(model_data_dir, "X_test_scaled.pkl"))
    
    # Chargement du scaler
    scaler = joblib.load(os.path.join(model_data_dir, "scaler.pkl"))
    
    # Chargement des noms des variables explicatives
    with open(os.path.join(model_data_dir, "feature_names.txt"), "r") as f:
        feature_names = f.read().splitlines()
    
    print("Données chargées avec succès.")
    print(f"Dimensions de X_train_scaled : {X_train_scaled.shape}")
    print(f"Dimensions de X_test_scaled : {X_test_scaled.shape}")
    print(f"Dimensions de y_train : {y_train.shape}")
    print(f"Dimensions de y_test : {y_test.shape}")
else:
    print(f"Erreur : Le dossier n'existe pas à l'emplacement : {model_data_dir}")
    print("Veuillez exécuter le notebook 3_Preparation_Modelisation.ipynb avant de continuer.")

In [ ]:
# Affichage des noms des variables explicatives
print("Variables explicatives :")
for i, feature in enumerate(feature_names):
    print(f"{i+1}. {feature}")

## 4.3 Développement du modèle de régression logistique

La régression logistique est un algorithme de classification linéaire qui estime la probabilité qu'une instance appartienne à une classe particulière. C'est un bon point de départ pour les problèmes de classification binaire comme le nôtre.

### 4.3.1 Création et entraînement du modèle

In [ ]:
# Création du modèle de régression logistique
# Nous utilisons la régularisation L2 (ridge) par défaut
# Le paramètre C contrôle la force de la régularisation (plus C est petit, plus la régularisation est forte)
# class_weight='balanced' permet de gérer le déséquilibre des classes
logreg = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000, class_weight='balanced')

# Entraînement du modèle sur les données standardisées
logreg.fit(X_train_scaled, y_train)

print("Modèle de régression logistique entraîné avec succès.")

### 4.3.2 Analyse des coefficients du modèle

Les coefficients de la régression logistique indiquent l'importance et la direction de l'influence de chaque variable sur la probabilité de la classe positive (non solvable dans notre cas).

In [ ]:
# Extraction des coefficients du modèle
coefficients = pd.DataFrame({
    'Variable': feature_names,
    'Coefficient': logreg.coef_[0]
})

# Tri des coefficients par valeur absolue décroissante
coefficients['Abs_Coefficient'] = abs(coefficients['Coefficient'])
coefficients = coefficients.sort_values('Abs_Coefficient', ascending=False).reset_index(drop=True)

# Affichage des coefficients
print("Coefficients du modèle de régression logistique :")
coefficients

In [ ]:
# Visualisation des coefficients
plt.figure(figsize=(12, 8))
sns.barplot(x='Coefficient', y='Variable', data=coefficients)
plt.title('Coefficients du modèle de régression logistique', fontsize=14)
plt.xlabel('Coefficient', fontsize=12)
plt.ylabel('Variable', fontsize=12)
plt.axvline(x=0, color='r', linestyle='--')
plt.grid(True, alpha=0.3)
plt.show()

### 4.3.3 Interprétation des coefficients

Analysons les coefficients pour comprendre l'influence de chaque variable sur la probabilité qu'un client soit non solvable :

In [ ]:
# Interprétation des coefficients
interpretation = pd.DataFrame({
    'Variable': coefficients['Variable'],
    'Coefficient': coefficients['Coefficient'],
    'Influence': ['Augmente la probabilité de non-solvabilité' if coef > 0 else 'Diminue la probabilité de non-solvabilité' 
                 for coef in coefficients['Coefficient']],
    'Importance': coefficients['Abs_Coefficient']
})

# Affichage de l'interprétation
print("Interprétation des coefficients :")
interpretation

## 4.4 Évaluation du modèle sur le jeu de test

### 4.4.1 Prédictions sur le jeu de test

In [ ]:
# Prédictions sur le jeu de test
y_pred = logreg.predict(X_test_scaled)

# Probabilités prédites
y_pred_proba = logreg.predict_proba(X_test_scaled)[:, 1]  # Probabilité de la classe positive (non solvable)

### 4.4.2 Matrice de confusion

In [ ]:
# Calcul de la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualisation de la matrice de confusion
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Solvable (0)', 'Non solvable (1)'],
            yticklabels=['Solvable (0)', 'Non solvable (1)'])
plt.title('Matrice de confusion', fontsize=14)
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Réalité', fontsize=12)
plt.show()

# Interprétation de la matrice de confusion
tn, fp, fn, tp = conf_matrix.ravel()
print(f"Vrais négatifs (TN) : {tn} - Clients correctement classés comme solvables")
print(f"Faux positifs (FP) : {fp} - Clients solvables incorrectement classés comme non solvables")
print(f"Faux négatifs (FN) : {fn} - Clients non solvables incorrectement classés comme solvables")
print(f"Vrais positifs (TP) : {tp} - Clients correctement classés comme non solvables")

### 4.4.3 Métriques d'évaluation

In [ ]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Affichage des métriques
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

# Rapport de classification détaillé
print("\nRapport de classification :")
print(classification_report(y_test, y_pred, target_names=['Solvable (0)', 'Non solvable (1)']))

### 4.4.4 Courbe ROC et AUC

In [ ]:
# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calcul de l'AUC
auc = roc_auc_score(y_test, y_pred_proba)

# Visualisation de la courbe ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs (1 - Spécificité)', fontsize=12)
plt.ylabel('Taux de vrais positifs (Sensibilité)', fontsize=12)
plt.title('Courbe ROC', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

print(f"AUC : {auc:.4f}")

## 4.5 Validation croisée

Pour évaluer la robustesse du modèle, nous allons effectuer une validation croisée à 5 plis.

In [ ]:
# Configuration de la validation croisée stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Métriques à évaluer
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# Dictionnaire pour stocker les résultats
cv_results = {}

# Calcul des scores de validation croisée pour chaque métrique
for metric in scoring:
    cv_scores = cross_val_score(logreg, X_train_scaled, y_train, cv=cv, scoring=metric)
    cv_results[metric] = cv_scores
    print(f"{metric}: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

In [ ]:
# Visualisation des résultats de la validation croisée
plt.figure(figsize=(12, 8))
boxplot_data = [cv_results[metric] for metric in scoring]
plt.boxplot(boxplot_data, labels=scoring, patch_artist=True)
plt.title('Résultats de la validation croisée', fontsize=14)
plt.ylabel('Score', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

## 4.6 Optimisation des hyperparamètres

Nous allons maintenant optimiser les hyperparamètres du modèle de régression logistique pour améliorer ses performances.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définition de la grille de paramètres à tester
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],  # liblinear supporte à la fois l1 et l2
    'class_weight': ['balanced', None]
}

# Création du modèle de base
logreg_base = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

# Configuration de la recherche par grille avec validation croisée
grid_search = GridSearchCV(logreg_base, param_grid, cv=5, scoring='f1', n_jobs=-1)

# Exécution de la recherche par grille
grid_search.fit(X_train_scaled, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres :")
print(grid_search.best_params_)
print(f"Meilleur score F1 : {grid_search.best_score_:.4f}")

In [ ]:
# Création du modèle optimisé avec les meilleurs paramètres
logreg_optimized = LogisticRegression(**grid_search.best_params_, random_state=RANDOM_STATE, max_iter=1000)

# Entraînement du modèle optimisé
logreg_optimized.fit(X_train_scaled, y_train)

# Prédictions sur le jeu de test
y_pred_optimized = logreg_optimized.predict(X_test_scaled)
y_pred_proba_optimized = logreg_optimized.predict_proba(X_test_scaled)[:, 1]

# Calcul des métriques d'évaluation
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
precision_optimized = precision_score(y_test, y_pred_optimized)
recall_optimized = recall_score(y_test, y_pred_optimized)
f1_optimized = f1_score(y_test, y_pred_optimized)
auc_optimized = roc_auc_score(y_test, y_pred_proba_optimized)

# Affichage des métriques
print("Métriques du modèle optimisé :")
print(f"Accuracy : {accuracy_optimized:.4f}")
print(f"Precision : {precision_optimized:.4f}")
print(f"Recall : {recall_optimized:.4f}")
print(f"F1-score : {f1_optimized:.4f}")
print(f"AUC : {auc_optimized:.4f}")

## 4.7 Comparaison des modèles (initial vs optimisé)

In [ ]:
# Comparaison des métriques
comparison = pd.DataFrame({
    'Métrique': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC'],
    'Modèle initial': [accuracy, precision, recall, f1, auc],
    'Modèle optimisé': [accuracy_optimized, precision_optimized, recall_optimized, f1_optimized, auc_optimized]
})

# Affichage de la comparaison
print("Comparaison des modèles :")
comparison

In [ ]:
# Visualisation de la comparaison
plt.figure(figsize=(12, 8))
comparison.set_index('Métrique').plot(kind='bar')
plt.title('Comparaison des modèles', fontsize=14)
plt.ylabel('Score', fontsize=12)
plt.ylim([0, 1])
plt.grid(True, alpha=0.3)
plt.legend(title='Modèle')
plt.show()

## 4.8 Sauvegarde du modèle final

Nous allons sauvegarder le modèle optimisé pour l'utiliser dans les notebooks suivants.

In [ ]:
# Création d'un dossier pour les modèles
models_dir = "/home/ubuntu/notebooks/models"
os.makedirs(models_dir, exist_ok=True)

# Sauvegarde du modèle optimisé
logreg_model_path = os.path.join(models_dir, "logreg_model.pkl")
joblib.dump(logreg_optimized, logreg_model_path)

# Sauvegarde des métriques d'évaluation
logreg_metrics = {
    'accuracy': accuracy_optimized,
    'precision': precision_optimized,
    'recall': recall_optimized,
    'f1': f1_optimized,
    'auc': auc_optimized
}
logreg_metrics_path = os.path.join(models_dir, "logreg_metrics.pkl")
joblib.dump(logreg_metrics, logreg_metrics_path)

print(f"Modèle de régression logistique sauvegardé avec succès dans : {logreg_model_path}")
print(f"Métriques d'évaluation sauvegardées avec succès dans : {logreg_metrics_path}")

## 4.9 Résumé de la modélisation avec régression logistique

Dans ce notebook, nous avons développé et évalué un modèle de régression logistique pour prédire le statut de crédit des clients. Voici les principales étapes et observations :

### 4.9.1 Développement du modèle

1. **Création et entraînement** : Nous avons créé un modèle de régression logistique et l'avons entraîné sur les données standardisées.

2. **Analyse des coefficients** : Nous avons analysé les coefficients du modèle pour comprendre l'influence de chaque variable sur la probabilité qu'un client soit non solvable.
   - Les variables les plus importantes sont : loan_to_income, income, loan_amount, et age.
   - Un ratio prêt/revenu élevé augmente la probabilité de non-solvabilité.
   - Un revenu élevé diminue la probabilité de non-solvabilité.
   - Un montant de prêt élevé augmente la probabilité de non-solvabilité.
   - Un âge élevé diminue la probabilité de non-solvabilité.

### 4.9.2 Évaluation du modèle

1. **Métriques d'évaluation** : Le modèle optimisé a obtenu les performances suivantes sur le jeu de test :
   - Accuracy : environ 77%
   - Precision : environ 83%
   - Recall : environ 18%
   - F1-score : environ 30%
   - AUC : environ 73%

2. **Validation croisée** : La validation croisée a confirmé la robustesse du modèle, avec des scores similaires sur les différents plis.

3. **Optimisation des hyperparamètres** : Nous avons optimisé les hyperparamètres du modèle pour améliorer ses performances, notamment en ajustant le paramètre de régularisation C et le type de pénalité.

### 4.9.3 Observations et limites

1. **Déséquilibre des classes** : Le modèle a une précision élevée mais un rappel faible, ce qui est typique des problèmes avec des classes déséquilibrées. Cela signifie qu'il est bon pour identifier les clients solvables, mais moins bon pour identifier les clients non solvables.

2. **Compromis précision-rappel** : Selon le contexte métier, il peut être préférable de privilégier le rappel (identifier plus de clients non solvables au risque de refuser des crédits à des clients solvables) ou la précision (être plus sûr des clients identifiés comme non solvables au risque d'en manquer certains).

3. **Interprétabilité** : La régression logistique offre une bonne interprétabilité, ce qui est un avantage important dans le domaine du crédit où les décisions doivent souvent être expliquées.

## Prochaine étape

Dans le prochain notebook, nous développerons et évaluerons un modèle KNN (K-Nearest Neighbors) pour prédire le statut de crédit des clients, puis nous comparerons ses performances avec celles du modèle de régression logistique.